# Contents
[Util](#1) <br>
- imports
- constants
- lighten_hex_color: return hex ID of lighter version of hex color
- choose_random_color: return random color from hard-coded palette
- glyph_type_color: return color corresponding to feature type in hard-coded dictionary
- center_feature: return centerpoint of feature

[Read and process data](#2) <br>
- gff_to_db: create db in current directory
- get_gene_exons: return dict mapping name of transcript to list of (start,end) of corresponding exons
- get_gene_UTR_lens:
- get_introns_from_exons:
- get_variants_bed:
- get_variants_maf:
- get_variants_vcf:

[Create glyphs](#3) <br>
- add_variation_glyph:
- add_intron_glyph:
- add_exon_glyph:
- add_UTR_glyph:
- add_domain_glyph:

[Create plots](#4) <br>
- constraint_view_plot:
- constraint_view:

<a id='1'></a>
### Util

In [3]:
import sys
sys.path.append("../utilities")
sys.path.append("../predict-constraint")

In [4]:
from compute_mutation_counts import compute_mutation_counts
import vizcoords
import gffutils
import numpy as np
from bokeh.io import show, save
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row, gridplot
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Rect, Circle, Range1d, HoverTool, GlyphRenderer, Segment, Line, MultiPolygons, VArea
from bokeh.models import CheckboxGroup, CustomJS, Legend
import random
import json
import math
import time

In [50]:
#constants
data_path = '/scratch/ucgd/lustre-work/quinlan/u6038618/constraint-tools/data/'
gff_path = data_path + 'Homo_sapiens.GRCh38.104.gff3' #feature data
gff_db_path = data_path + 'Homo_sapiens.GRCh38.104.gff3.db'
bed_path = data_path +'gnomad_v3_variants.bed' #variant data
gtf_db_path = data_path + 'pfam.sorted.gtf.db' #domain data
model_path = data_path + 'model.json' #input for mutation counts
gene_name = 'KCNQ2'

In [16]:
#bokeh will output to notebook rather than opening new tab in browser
output_notebook()

Loading BokehJS ...

In [17]:
#param color: str of form '#xxxxxx' or 'xxxxxx'; color to be lightened
#param delta: int 0-100; amount to lighten color by
#return new_color: str of form '#xxxxxx'; lightened hex color
def lighten_hex_color(color, delta):
    def limit(x):
        if x > 255: return 255
        if x < 0: return 0
        return x

    if '#' in color: color = color[1:]
    
    r = hex(limit(int(color[:2], 16) + delta))[2:]
    g = hex(limit(int(color[2:4], 16) + delta))[2:]
    b = hex(limit(int(color[4:6], 16) + delta))[2:]
    
    new_color = '#{}{}{}'.format(r,g,b)
    return new_color

In [18]:
#return random_color: str of form '#xxxxxx'; random hex color from hard-coded palette
def choose_random_color():
    palette = np.unique(['#f94144', '#f3722c', '#f8961e', '#f9c74f', '#90be6d', '#43aa8b', '#577590', '#a53860', '#7161ef',
              '#bf99f2', '#49a078', '#f7b538', '#5fad56', '#4d9078', '#b4436c', '#404e7c', '#982649', '#104547', 
              '#7E7F9A', '#90C290', '#809848', '#D90368', '#E899DC', '#613DC1', '#8FC93A', '#00916E', '#E086D3', 
               '#861388','#AB81CD', '#3C787E', '#D36135', '#5B2333', '#49A078', '#98DFAF', '#29524A', '#D6C2EA', 
               '#5C8001', '#FF9FE5', '#FF8C42', '#6BA292', '#A09BE7', '#FA9500', '#E0BE36', '#993955', 
               '#BF3100', '#EC9F05', '#F5BB00'])
    
    random_color = random.choice(palette)
    return random_color

In [108]:
#param feature_type: str; input can be 'variant','intron','exon','UTR'
#return glyph_color: str of form '#xxxxxx'; color corresponding to feature type in hard-coded dict
def glyph_type_color(feature_type):
    glyph_color_dict=dict(
        variant= '#ee6055',
        intron = '#005f73',
        exon = '#4ecdc4',
        UTR = '#105c56',
        expected_mutation = lighten_hex_color('#593959',70),
        observed_mutation = lighten_hex_color('#8B80F9',50),
        coverage = '#94d499',)
    glyph_color = glyph_color_dict[feature_type]
    return glyph_color

In [20]:
#param feature: (int, int); start and end of feature
#return center: int; center point of feature
def center_feature(feature):
    center = feature[0] + (feature[1] - feature[0])/2
    return center

<a id='2'></a>
### Read and process data

In [21]:
#param gff: filepath to .gff
#behavior: create db in current directory
def gff_to_db(gff_path):
    gffutils.create_db(gff, dbfn='{}.db'.format(gff), force=True, keep_order=True, merge_strategy='merge', sort_attribute_values=True)

In [22]:
#only do this once
#gff_to_db('Homo_sapiens.GRCh38.104.gff3')

In [23]:
#only do this once
#gff_to_db('pfam.sorted.gtf')

In [24]:
def get_domains(gtf_db_path, start=-1, end=-1, seqid='chr20', strand='-'):
    domains = []
    db = gffutils.FeatureDB(gtf_db_path, keep_order=True)
    for s in list(db.region(seqid=seqid, featuretype='exon', strand=strand)):
        domain_dict = dict(start=s.start, end=s.end, name=s['gene_id'][0]) #more than one gene name?
        domains.append(domain_dict)
    
    if start >= 0: domains = [d for d in domains if d['start'] >= start]
    if end >= 0: domains = [d for d in domains if d['end'] <= end]
    return domains

In [25]:
#param gff_db_path: filepath to database generated by gff_to_db
#gene_name: gene to get exons from
#return exon_dict: {str->[(int,int)]}; map name of transcript to list of (start,end) of corresponding exons
def get_gene_exons(gff_db_path, gene_name):
    db = gffutils.FeatureDB(gff_db_path, keep_order=True)
    feat_ls = []
    
    #get ID from name
    for f in db.all_features(featuretype='gene'):
        try:
            if gene_name in f['Name'][0]: feat_ls.append(f)
        except: continue
    
    if len(feat_ls) > 1:
        print('More than one gene with name {}'.format(gene_name))
        return
    
    feat = feat_ls[0]  
    chr_num = feat.seqid
    exon_dict = {}
    
    #get all exons and flatten
    exons_all = db.children(feat, featuretype='exon')
    
    exon_coords_all = [(e.start, e.end) for e in exons_all]
    #flattened exons strands set to [] because I don't want to deal with that right now
    exon_dict['flattened-exons'] = dict(true_coords=vizcoords.flatten_exons(exon_coords_all),
                                       direction='', chr_num=chr_num)
    
    #indiv mRNAs
    transcripts = list(db.children(feat, featuretype='mRNA'))
    for t in transcripts:
        exons = list(db.children(t, featuretype='exon'))
        strands = [e.strand for e in exons]        
        exon_coords = [(e.start, e.end) for e in exons]
        direction = t.strand
        tname = t['Name'][0]
        exon_dict[tname] = dict(true_coords=exon_coords, direction=direction, chr_num=chr_num)
    return exon_dict

In [26]:
exon_dict=get_gene_exons(gff_db_path,gene_name)

In [180]:
#param gff_db_path: filepath to database generated by gff_to_db
#gene_name: gene to get exons from
#return exon_dict: {str->[(int,int)]}; map name of transcript to list of (start,end) of corresponding exons
def get_transcript_dict(gff_db_path, gene_name):
    db = gffutils.FeatureDB(gff_db_path, keep_order=True)
    feat_ls = []
    
    #get ID from name
    for f in db.all_features(featuretype='gene'):
        try:
            if gene_name in f['Name'][0]: feat_ls.append(f)
        except: continue
    
    if len(feat_ls) > 1:
        print('More than one gene with name {}'.format(gene_name))
        return
    
    feat = feat_ls[0]  
    chr_num = feat.seqid
    transcript_dict = {}
    
    #get all exons and flatten
    exons_all = db.children(feat, featuretype='exon')
    
    exon_coords_all = [(e.start, e.end) for e in exons_all]
    transcript_dict['flattened-exons'] = dict(chr_num=chr_num, true_coords=vizcoords.flatten_exons(exon_coords_all),
                                       direction='', tUTR_len=-1, fUTR_len=-1)
    
    #indiv mRNAs
    transcripts = list(db.children(feat, featuretype='mRNA'))
    for t in transcripts:
        exons = list(db.children(t, featuretype='exon'))
        strands = [e.strand for e in exons]        
        exon_coords = [(e.start, e.end) for e in exons]
        direction = t.strand
        tname = t['Name'][0]
        ##UTR lens
        try:
            fUTR = list(db.children(t, featuretype='five_prime_UTR'))[0]
            fUTR_len = fUTR.end - fUTR.start + 1
        except: fUTR_len = -1
        try:
            tUTR = list(db.children(t, featuretype='three_prime_UTR'))[0]
            tUTR_len = tUTR.end - tUTR.start + 1
        except: tUTR_len = -1
    
        transcript_dict[tname] = dict(chr_num=chr_num, true_coords=exon_coords, direction=direction, 
                                tUTR_len=tUTR_len, fUTR_len=fUTR_len)
    return transcript_dict

In [181]:
def get_gene_UTR_lens(gff_db_path, gene_name):
    db = gffutils.FeatureDB(gff_db_path, keep_order=True)
    feat_ls = []

    #get ID from name
    for f in db.all_features(featuretype='gene'):
        try:
            if gene_name in f['Name'][0]: feat_ls.append(f)
        except: continue
    
    if len(feat_ls) > 1:
        print('More than one gene with name {}'.format(gene_name))
        return
    
    feat = feat_ls[0]    
    UTR_dict = {}
    UTR_dict['flattened-exons'] = (-1, -1)
    
    transcripts = db.children(feat, featuretype='mRNA')
    
    for t in transcripts:
        tname = t['Name'][0]
        
        try:
            fUTR = list(db.children(t, featuretype='five_prime_UTR'))[0]
            fUTR_len = fUTR.end - fUTR.start + 1
        except:
            fUTR_len = -1
        
        try:
            tUTR = list(db.children(t, featuretype='three_prime_UTR'))[0]
            tUTR_len = tUTR.end - tUTR.start + 1
        except:
            tUTR_len = -1
        
        UTR_dict[tname] = (fUTR_len, tUTR_len)

    return UTR_dict

In [163]:
get_gene_UTR_lens(gff_db_path, gene_name)

{'flattened-exons': (-1, -1),
 'KCNQ2-201': (192, 67),
 'KCNQ2-202': (46, 178),
 'KCNQ2-203': (-1, 407),
 'KCNQ2-204': (192, 6436),
 'KCNQ2-205': (42, 198),
 'KCNQ2-207': (135, 568),
 'KCNQ2-209': (92, 568),
 'KCNQ2-211': (-1, 227),
 'KCNQ2-213': (214, 6434),
 'KCNQ2-214': (-1, -1),
 'KCNQ2-215': (92, 568),
 'KCNQ2-216': (-1, 5),
 'KCNQ2-217': (-1, 220),
 'KCNQ2-218': (92, 369),
 'KCNQ2-227': (-1, -1),
 'KCNQ2-236': (5, 565)}

In [30]:
def get_introns_from_exons(exons):
    introns = []
    
    flat_exons = [item for sublist in exons for item in sublist][:-1]
    if len(flat_exons) % 2 == 1: flat_exons = [0] + flat_exons
        
    for i in range(0,len(flat_exons)-1,2):
        introns.append((flat_exons[i]+1,flat_exons[i+1]-1))
    
    return introns

In [31]:
def get_variants_bed(bed_path, gene_name=''):
    with open(bed_path) as fh:
        variant_lines = fh.readlines()[1:] #remove header
        variant_ls = []
        
        for line in variant_lines:
            line_ls = line.split('\t')
            variant_dict = dict(pos=int(line_ls[1]), ref=line_ls[3], alt=line_ls[4], 
                                count=int(line_ls[5]), gene_name=line_ls[6])
            variant_ls.append(variant_dict)
            
    if len(gene_name) > 0:
        variant_ls = [variant for variant in variant_ls if gene_name in variant['gene_name']]
    
    return variant_ls

In [32]:
def get_variants_maf(maf_path):
    f = open(maf_path,'r')
    fi = f.readlines()
    f.close()
    variation_ls = []

    fi_gene = [line for line in fi if 'KCNQ2' in line]
    for line in fi_gene[:100]:
        line_ls = line.split('\t')
        
        variation_dict = dict(ref_allele = line_ls[7], alt_allele = line_ls[9],
        start = int(line_ls[2]), end = int(line_ls[3]),
        alt_count = int(line_ls[37]), ref_count = int(line_ls[38]))
        
        variation_ls.append(variation_dict)
    return variation_ls

In [33]:
#param vcf_path: str; path to .vcf
#param start: int; return variants starting from this coord (if negative, start at earliest coord)
#param end: int; return variants ending at this coord (if negative, end at latest coord)
#return variant_ls: [{str->strORint}]; list of dictionaries corresponding to variants, {field->value}
def get_variants_vcf(vcf_path, start=-1, end=-1):
    with open(vcf_path) as fh:
        variant_lines = [line for line in fh.readlines() if line[0] != '#'] #remove headers
        variant_ls = [] #will hold variant dictionaries
        for line in variant_lines:
            line_ls = line.split('\t')
            variant_dict = dict(pos=int(line_ls[1]), ref=line_ls[3], alt=line_ls[4])
            variant_ls.append(variant_dict)
    if start >= 0: 
        variant_ls = [variant for variant in variant_ls if variant['pos'] > start]
    if end >= 0: variant_ls = [variant for variant in variant_ls if variant['pos'] < end]
        
    return variant_ls

<a id='3'></a>
### Create glyphs

In [34]:
def add_variation_glyph(plot, vars_compact, vars_original, counts, max_count, height=9, color='', radius=5): #height prev 18
    
#     interval = 9/max_count #plot_height = 120
    interval = 9/max_count #plot height = 200
    x = vars_compact
#     y0_segment = [10 for v in vars_compact] #plot_height = 120
    y0_segment = [5 for v in vars_compact] #plot_height = 200
    y1_segment = [height+interval*(c-1) for c in counts]
    y_circle = [height+interval*(c-1) for c in counts]
    r_circle = [radius for v in vars_compact]
    info = ['info' for v in vars_compact]
    
    source = ColumnDataSource(dict(x=x, y0_segment=y0_segment, y1_segment=y1_segment, y_circle=y_circle, r_circle=r_circle, true_pos=vars_original, counts=counts, info=info))
    
    if color == '': color = glyph_type_color('variant')
    hover_color = lighten_hex_color(color, 40)
    
    segment_glyph = Segment(x0='x', y0='y0_segment', x1='x', y1='y1_segment', line_color=color, line_width=3)
    segment_hover_glyph = Segment(x0='x', y0='y0_segment', x1='x', y1='y1_segment', line_color=hover_color, line_width=3)

    circle_glyph = Circle(x="x", y='y_circle', radius='r_circle', radius_units='screen', fill_color='white', line_color=color, line_width=3.5)
    circle_hover_glyph = Circle(x="x", y='y_circle', radius='r_circle', radius_units='screen', fill_color='white', line_color=hover_color, line_width=3.5)

    segment_glyph = plot.add_glyph(source, segment_glyph, hover_glyph=segment_hover_glyph)
    circle_glyph = plot.add_glyph(source, circle_glyph, hover_glyph=circle_hover_glyph)
    
    return segment_glyph,circle_glyph 

In [35]:
def add_intron_glyph(plot, features_compact, features_original, color='#005f73', fill_alpha=1, height=17, width=-1, transcript_len=-1):
    x = [center_feature(f) for f in features_compact]
    y = [5 for f in features_compact] #prev 10
    w = [width for f in features_compact] #avoid white spaces
    h = [height for f in features_compact]
    
    ###METADATA
    feat_type = ['Intron' for f in features_compact]
    adj_start = [start for (start,end) in features_compact]
    adj_end = [end for (start,end) in features_compact]
    true_start = [start for (start,end) in features_original]
    true_end = [end for (start,end) in features_original]
    true_len = [end-start+1 for (start,end) in features_original] #+1 for 1 indexed coords
    ###
    
    source = ColumnDataSource(dict(x=x, y=y, w=w, h=h,
                                   feat_type=feat_type,
                                   adj_start=adj_start, adj_end=adj_end,
                                   true_len=true_len, true_start=true_start, true_end=true_end))
    
    hover_color = lighten_hex_color(color, 50)
    
    glyph = Rect(x='x', y='y', width='w', height='h', height_units='screen', fill_color=color, fill_alpha=fill_alpha, line_color=color)
    hover_glyph = Rect(x='x', y='y', width='hover_w', height='hover_h', height_units='screen', fill_color=hover_color, fill_alpha=fill_alpha, line_color=hover_color)

    return plot.add_glyph(source,glyph,hover_glyph=hover_glyph)

In [36]:
def add_exon_glyph(plot, features_compact, features_original, direction, color='#4ecdc4', height=17, transcript_len=-1):
    x = [center_feature(f) for f in features_compact]
#     y = [10 for f in features_compact] #plot_height = 120
    y = [5 for f in features_compact] #plot_height=200
    w = [end-start for (start,end) in features_compact]
    h = [height for f in features_compact]
    
    #METADATA
    feat_type = ['Exon' for f in features_compact]
    adj_start = [start for (start,end) in features_compact]
    adj_end = [end for (start,end) in features_compact]
    true_start = [start for (start,end) in features_original]
    true_end = [end for (start,end) in features_original]
    true_len = [end-start+1 for (start,end) in features_original] #+1 for 1 indexed coords
    
    exon_source = ColumnDataSource(dict(x=x, y=y, w=w, h=h,
                                   feat_type=feat_type,
                                   adj_start=adj_start, adj_end=adj_end,
                                   true_len=true_len, true_start=true_start, true_end=true_end))
    
    hover_color = lighten_hex_color(color, 20)
    
    exon_glyph = Rect(x='x', y='y', width='w', height='h', height_units='screen', fill_color=color, line_color=color)
    hover_glyph = Rect(x='x', y='y', width='hover_w', height='hover_h', height_units='screen', fill_color=hover_color, line_color=hover_color)
    
    if direction=='': 
        exon_glyph = plot.add_glyph(exon_source,exon_glyph,hover_glyph=hover_glyph)
        return exon_glyph,None
    
    #ADD ARROWS#
    def adjust_arrow_coords(coords,direction):
        ret = [j*(adj_end[i]-adj_start[i])/100 for j in coords]
        for k in range(3): #do this in 3 rounds so the smaller arrows don't get too small
            if (max(ret) - min(ret))/transcript_len > 0.01:
                ret=[r*.4 for r in ret]
        if direction=='-': ret = [-r for r in ret]
        return [r+x[i] for r in ret]

    xs_dict = []
    for i in range(len(features_compact)):

        xs_dict.append([{'exterior': adjust_arrow_coords([-20, -20, 0, 0],direction), 'holes': []}])
        xs_dict.append([{'exterior': adjust_arrow_coords([0, 0, 20],direction), 'holes': []}])

    ys_dict = []
    for i in range(len(features_compact)):
        ys_dict.append([{'exterior': [4.5, 5.5, 5.5, 4.5], 'holes': []}]) #rect: prev [9, 11, 11, 9]
        ys_dict.append([{'exterior': [3.5, 6.5, 5], 'holes': []}]) #triangle: prev [7,13,10]

    xs = [[[p['exterior'], *p['holes']] for p in mp] for mp in xs_dict]
    ys = [[[p['exterior'], *p['holes']] for p in mp] for mp in ys_dict]

    arrow_source = ColumnDataSource(dict(xs=xs, ys=ys))
    arrow_glyph = MultiPolygons(xs='xs', ys='ys', fill_color='#39b3aa', line_color='#39b3aa')
    arrow_glyph = plot.add_glyph(arrow_source, arrow_glyph)
    arrow_glyph.level = 'overlay'
    
    exon_glyph = plot.add_glyph(exon_source,exon_glyph,hover_glyph=hover_glyph)
    
    return exon_glyph,arrow_glyph

In [37]:
def add_UTR_glyph(plot, features_compact, features_original, feature_type='', color='#cab2d6', fill_alpha=0.4, height=17):
    x = [center_feature(f) for f in features_compact]
    y = [5 for f in features_compact] #prev 10
    w = [end-start for (start,end) in features_compact]
    h = [height for f in features_compact]
    hover_h = [height+15 for f in features_compact]
    hover_w = [i+10 for i in w]
    
    #metadata
    feat_type = [feature_type for f in features_compact]
    adj_start = [start for (start,end) in features_compact]
    adj_end = [end for (start,end) in features_compact]
    true_start = [start for (start,end) in features_original]
    true_end = [end for (start,end) in features_original]
    true_len = [end-start+1 for (start,end) in features_original] #+1 for 1 indexed coords
    
    source = ColumnDataSource(dict(x=x, y=y, w=w, h=h, hover_h=hover_h,
                                   feat_type=feat_type,
                                   adj_start=adj_start, adj_end=adj_end,
                                   true_len=true_len, true_start=true_start, true_end=true_end))
    
    color = glyph_type_color('UTR')    
    hover_color = lighten_hex_color(color, 40)
    
    glyph = Rect(x='x', y='y', width='w', height='h', height_units='screen', fill_color=color, fill_alpha=fill_alpha, line_color=color)
    hover_glyph = Rect(x='x', y='y', width='hover_w', height='hover_h', height_units='screen', fill_color=hover_color, fill_alpha=fill_alpha, line_color=hover_color)

    return plot.add_glyph(source,glyph,hover_glyph=hover_glyph)

In [38]:
def add_domain_glyph(plot, domains_compact, domains_original, domain_names, colors, height=19):
    x = [center_feature(f) for f in domains_compact]
    y = [0 for f in domains_compact]
    w = [end-start for (start,end) in domains_compact]
    h = [height for f in domains_compact]
    hover_colors = [lighten_hex_color(color,40) for color in colors]
    
    #metadata
    adj_start = [start for (start,end) in domains_compact]
    adj_end = [end for (start,end) in domains_compact]
    true_start = [start for (start,end) in domains_original]
    true_end = [end for (start,end) in domains_original]
    true_len = [end-start+1 for (start,end) in domains_original] #+1 for 1 indexed coords
    
    source = ColumnDataSource(dict(x=x, y=y, w=w, h=h, colors=colors, hover_colors=hover_colors,
                                   domain_names=domain_names,
                                   adj_start=adj_start, adj_end=adj_end,
                                   true_len=true_len, true_start=true_start, true_end=true_end))
    
    glyph = Rect(x='x', y='y', width='w', height='h', height_units='screen', fill_color='colors', line_color='colors')
    hover_glyph = Rect(x='x', y='y', width='hover_w', height='h', height_units='screen', fill_color='hover_colors', line_color='hover_colors')

    return plot.add_glyph(source,glyph,hover_glyph=hover_glyph)

In [40]:
#param xs: [int]; x values
#param ys: [int]; y values
#param y0: int/float; location of y axis with respect to plot
#param fill_area: bool; if True, fill area under line, else leave transparent
def add_line_glyph(plot, xs, ys, y0=0, fill_area=False, line_color='black', line_width=2, line_alpha=1):
    source = ColumnDataSource(dict(xs=xs, y1s=[y+y0 for y in ys], y0s=[y0 for i in range(len(ys))]))
    if fill_area:
        line_glyph = VArea(x='xs', y1='y0s', y2='y1s', fill_color=line_color, fill_alpha=line_alpha)
    else:
        line_glyph = Line(x='xs', y='y1s', line_color=line_color, line_width=line_width, line_alpha=line_alpha)
    return plot.add_glyph(source, line_glyph)

In [52]:
def add_constraint_glyph(plot, exon_coord_original, exon_coord_compact, chr_num, model=model_path, window_size=51, window_stride=25):
    ###within an exon, should be able to subtract a constant from windowPositions to get compact coords
    orig_start = exon_coord_original[0] - 1400000
    orig_end = exon_coord_original[1] - 1400000
    compact_start = exon_coord_compact[0]
    compact_end = exon_coord_compact[1]
    
    diff = orig_start - compact_start

    mutation_dict = compute_mutation_counts('{}:{}-{}'.format(chr_num,orig_start,orig_end), model, window_size, window_stride)
    xs = [x-diff for x in mutation_dict['windowPositions']]
    y1s = [y1*4 for y1 in mutation_dict['windowExpectedMutationCounts']]
    y2s = [y2*4 for y2 in mutation_dict['windowObservedMutationCounts']]
    
    expected_glyph = add_line_glyph(plot, xs, y1s, y0=8, line_color=glyph_type_color('expected_mutation'), line_alpha=0.6)
    observed_glyph = add_line_glyph(plot, xs, y2s, y0=8, line_color=glyph_type_color('observed_mutation'), line_alpha=0.7)
    
    return expected_glyph,observed_glyph,y1s,y2s

In [94]:
def smooth(xs, start, end, k):
    xs = [float(x) for x in xs]
    for i in range(0, len(xs), k):
        window_avg = np.mean(xs[i:i+k])
        print(window_avg)
        for j in range(i, i+k):
            if j < len(xs): xs[j] = float(window_avg)
    ys = list(range(start,end))
    

In [98]:
#param k: int; smoothing window
def add_coverage_glyph(plot, exon_coord_original, exon_coord_compact, k=1):
    def pois_cov():
        c = np.random.poisson(9, 1)[0]/10
        while c < 0 or c > 1:
            c = np.random.poisson(9, 1)[0]/10
        return c
    xs = list(range(exon_coord_compact[0],exon_coord_compact[1]+1))
    ys = []
    for x in xs:
        ys.append(pois_cov()*20)
    
    #smooth ys
    for i in range(0, len(ys), k):
        window_avg = np.mean(ys[i:i+k])
        for j in range(i, i+k):
            if j < len(ys): ys[j] = float(window_avg)
        
    coverage_glyph = add_line_glyph(plot, xs, ys, y0=6, fill_area=True, line_color=glyph_type_color('coverage'), line_alpha=0.4)
    return coverage_glyph

<a id='4'></a>
### Create plots

In [158]:
def constraint_view_plot(glyph_dict, exon_coords, var_coords, var_counts, fUTR_len, tUTR_len, chr_num, domains=[], direction='', title=''):
###right now assuming one UTR at each end of the transcript, correctly annotated
    exons_compact,vars_compact = vizcoords.adjust_coordinates(exon_coords, var_coords)    
    intron_coords = get_introns_from_exons(exon_coords)
    introns_compact = get_introns_from_exons(exons_compact)
    
    domain_coords = []
    domain_names = []
    domain_colors = []
    domains_compact = []
    domains_original = []
    
    if len(domains) > 0:
        for di in domains:
            d_start = di['start']
            d_end = di['end']
            d_name = di['name']
            d_color = di['color']
            for idx,(e_start,e_end) in enumerate(exon_coords):
                if d_start >= e_start and d_end <= e_end:
                    domain_coords.append((d_start,d_end))
                    domain_names.append(d_name)
                    domain_colors.append(d_color)
                    domains_compact.append((exons_compact[idx][0] + d_start - e_start, 
                                           exons_compact[idx][1] - (e_end - d_end)))
                    domains_original.append((d_start,d_end))
    
#     plot_height = 120
    plot_height = 200
    
    plot = figure(title=title, plot_width=1500, plot_height=plot_height,min_border=0, toolbar_location=None,
               x_range=Range1d(0, exons_compact[-1][1]), y_range=Range1d(0,30), background_fill_color='white')
    plot.grid.grid_line_color = None
        
    #COVERAGE
    coverage_ls = []
    for i in range(len(exon_coords)):
        coverage_glyph = add_coverage_glyph(plot, exon_coords[i], exons_compact[i], k=25)
        coverage_ls.append(coverage_glyph)
        
    #CONSTRAINT
    max_y = -1
    expected_ls=[]
    observed_ls=[]
    for i in range(len(exon_coords)):
        expected_glyph,observed_glyph,y1s,y2s = add_constraint_glyph(plot,exon_coords[i], exons_compact[i],chr_num)   
        expected_ls.append(expected_glyph)
        observed_ls.append(observed_glyph)
        if max_y < max(y1s): max_y = max(y1s)
        if max_y < max(y2s): max_y = max(y2s)
    
    step=10
    orig_ticks = list(range(8, 31, step))
    constraint_ticks = list(range(0, max_y+step, step))
    orig_ticks = orig_ticks[:len(constraint_ticks)]
    ticker_dict = {}
    for idx in range(len(orig_ticks)):
        ticker_dict[orig_ticks[idx]] = str(constraint_ticks[idx]/4)
    
    plot.yaxis.ticker = orig_ticks
    plot.yaxis.major_label_overrides = ticker_dict
    
    glyph_dict['constraint_ls'] = expected_ls
    glyph_dict['observed_ls'] = observed_ls
    
    #VARIANTS
    if len(vars_compact) > 0: #avoid bokeh warning
        #indices of variations that don't fall in introns
        vars_idx = [i for i in range(len(var_coords)) if vizcoords.var_coord_in_exon_coords(var_coords[i], exon_coords)]
        vars_original = [var_coords[idx] for idx in vars_idx]
        max_count = max([var_counts[idx] for idx in vars_idx])
        var_counts = [var_counts[idx] for idx in vars_idx]        
    
        ray_glyph,circle_glyph = add_variation_glyph(plot, vars_compact, vars_original, var_counts, max_count)
        tooltips_variations = [('Position (adjusted)', '@x'), ('Position (true)', '@true_pos'), ('Count', '@counts'), ('Info', '@info')]
        plot.add_tools(HoverTool(tooltips=tooltips_variations, renderers=[circle_glyph,ray_glyph], point_policy='follow_mouse', attachment='below'))
    #
    else: 
        ray_glyph=None
        circle_glyph=None
    
    tooltips_features = [('Type','@feat_type'), ('Start (adjusted)', '@adj_start'), ('End (adjusted)', '@adj_end'), 
                      ('Start (true)', '@true_start'), ('End (true)', '@true_end'), ('Length', '@true_len')]
    
    glyph_dict['ray_glyph']
    
    #INTRONS    
    introns_glyph = add_intron_glyph(plot, introns_compact, intron_coords, height=14, width=15)
    plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[introns_glyph], point_policy='follow_mouse', attachment='below'))    
    
    #EXONS
    exons_glyph,arrow_glyph = add_exon_glyph(plot, exons_compact, exon_coords, direction, transcript_len=exons_compact[-1][1]-exons_compact[0][0])
    plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[exons_glyph], point_policy='follow_mouse', attachment='below'))

    #UTRs
    tUTR_glyph=None
    if tUTR_len >= 0:
        tUTR_glyph = add_UTR_glyph(plot, [(exons_compact[0][0], exons_compact[0][0] + tUTR_len)], [(exon_coords[0][0], exon_coords[0][0] + tUTR_len)], feature_type='3\' UTR', color='#105c56')
        plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[tUTR_glyph], point_policy='follow_mouse', attachment='below'))
    
    fUTR_glyph=None
    if fUTR_len >= 0:
        fUTR_glyph = add_UTR_glyph(plot, [(exons_compact[-1][1]-fUTR_len, exons_compact[-1][1])], [(exon_coords[-1][1]-fUTR_len, exon_coords[-1][1])], feature_type='5\' UTR', color='#105c56')
        plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[fUTR_glyph], point_policy='follow_mouse', attachment='below'))
        
    #DOMAINS
    tooltips_domains = [('Name','@domain_names'), ('Start (adjusted)', '@adj_start'), ('End (adjusted)', '@adj_end'), 
                      ('Start (true)', '@true_start'), ('End (true)', '@true_end'), ('Length', '@true_len')]
    domains_glyph = add_domain_glyph(plot, domains_compact, domains_original, domain_names, domain_colors)
    plot.add_tools(HoverTool(tooltips=tooltips_domains, renderers=[domains_glyph], point_policy='follow_mouse', attachment='below'))
    
    return plot,tUTR_glyph,fUTR_glyph,ray_glyph,circle_glyph,domains_glyph,arrow_glyph,expected_ls,observed_ls,coverage_ls

In [182]:
#right now vcf is just a list of ints, will change to process a .vcf file
#param gff_db_path: string;
#param vcf: ; 
#param gene_name: string;
#param transcript_IDs: [string];
def constraint_view(gff_db_path, bed_path, gene_name, transcript_IDs, pfam_db_path): 
    transcript_dict = get_transcript_dict(gff_db_path, gene_name) #only doing this once during dev
#     UTR_lens_dict = get_gene_UTR_lens(gff_db_path, gene_name)
    
    #BED
    variant_ls = get_variants_bed(bed_path, gene_name=gene_name)
    var_coords = [d['pos'] for d in variant_ls]
    var_counts = [d['count'] for d in variant_ls]
    
    #MAF
#     variant_ls = get_variants_maf(maf_path)
#     var_coords = [d['start'] for d in variant_ls]
#     var_counts = [d['alt_count'] for d in variant_ls]
    #VCF
    #for VCF, it doesn't seem to have count info, so right now counts are number of mutations types, e.g. C>T, C>A
#     exons_start = exon_coords_dict['flattened-exons'][0][0]
#     exons_end = exon_coords_dict['flattened-exons'][-1][1]
#     variant_ls = get_variants_vcf('cosmic_coding_muts_chr20.vcf', start=exons_start, end=exons_end)
#     var_coords,var_counts = np.unique([variant['pos'] for variant in variant_ls], return_counts=True)
    
    exons_start = exon_dict['flattened-exons']['true_coords'][0][0]
    exons_end = exon_dict['flattened-exons']['true_coords'][-1][1]
    domains = get_domains(pfam_db_path, start=exons_start, end=exons_end)
    
    domain_names = np.unique([d['name'] for d in domains])
    domain_color_dict = {}
    for name in domain_names:
        domain_color_dict[name] = choose_random_color()
    
    for idx in range(len(domains)):
        domains[idx]['color'] = domain_color_dict[domains[idx]['name']]
            
    plot_ls = []
    tUTR_ls = []
    fUTR_ls = []
    ray_ls = []
    circle_ls = []
    domains_ls = []
    arrow_ls = []
    expected_ls_flat = []
    observed_ls_flat = []
    coverage_ls_flat = []
    
    if transcript_IDs == 'all': transcript_IDs = list(exon_dict.keys())
    
    for idx,ID in enumerate(transcript_IDs):
        print(ID)
        title = 'gene={}; transcript={}'.format(gene_name, ID)
        plot,tUTR_glyph,fUTR_glyph,ray_glyph,circle_glyph,domains_glyph,arrow_glyph,expected_ls,observed_ls,coverage_ls = constraint_view_plot(transcript_dict[ID]['true_coords'], var_coords, var_counts, 
                                                          transcript_dict[ID]['tUTR_len'], transcript_dict[ID]['fUTR_len'], chr_num=transcript_dict[ID]['chr_num'], 
                                                          domains=domains, direction=transcript_dict[ID]['direction'], title=title)
        plot_ls.append(plot)
        tUTR_ls.append(tUTR_glyph)
        fUTR_ls.append(fUTR_glyph)
        ray_ls.append(ray_glyph)
        circle_ls.append(circle_glyph)
        domains_ls.append(domains_glyph)
        expected_ls_flat.extend(expected_ls)
        observed_ls_flat.extend(observed_ls)
        coverage_ls_flat.extend(coverage_ls)
        arrow_ls.append(arrow_glyph)
            
    legend_plot = figure(plot_height=70,plot_width=225,toolbar_location=None)
    legend_plot.yaxis.visible = legend_plot.xaxis.visible = legend_plot.grid.visible = False
    legend_glyph_expected = legend_plot.line(x=[0,1],y=[0,1],line_color=glyph_type_color('expected_mutation'))
    legend_glyph_observed = legend_plot.line(x=[0,1],y=[0,1],line_color=glyph_type_color('observed_mutation'))
    legend = Legend(items=[('Expected mutation counts', [legend_glyph_expected]), ('Observed mutation counts', [legend_glyph_observed])], location=(0,0))
    legend_glyph_expected.visible=False
    legend_glyph_observed.visible=False
    legend_plot.add_layout(legend, 'center')
    
    checkbox = CheckboxGroup(labels=['Show UTRS', 'Show domains', 'Show direction of translation', 'Show variants', 'Show mutation counts', 'Show sequencing coverage'],
                             active=[0, 1, 2, 3, 4, 5], width=100)

    callback = CustomJS(args=dict(UTR=tUTR_ls+fUTR_ls, rays=ray_ls, circles=circle_ls, domains=domains_ls, arrows=arrow_ls, expected=expected_ls_flat, observed=observed_ls_flat, coverage=coverage_ls), code='''
        function setVis(glyphs, vis){
            for (let i = 0; i < glyphs.length; i++){
                if(glyphs[i] != null){
                    glyphs[i].visible = vis
                }
            }
        }
        if (cb_obj.active.includes(0)) {setVis(UTR,true)}
        else {setVis(UTR,false)}
        if (cb_obj.active.includes(1)) {setVis(domains,true)}
        else {setVis(domains,false)}
        if (cb_obj.active.includes(2)) {setVis(arrows,true)}
        else {setVis(arrows,false)}
        if (cb_obj.active.includes(3)) {setVis(rays,true); setVis(circles,true)}
        else {setVis(rays,false); setVis(circles,false)}
        if (cb_obj.active.includes(4)) {setVis(expected,true); setVis(observed,true)}
        else {setVis(expected,false); setVis(observed,false)}
        if (cb_obj.active.includes(5)) {setVis(coverage,true)}
        else {setVis(coverage,false)}
        
        
    ''')

    checkbox.js_on_change('active', callback)
    show(column([checkbox,legend_plot] + [p for p in plot_ls]))    #TODO, option for inc. constraint or not
    #save(plot_ls[0],'test.html')

In [202]:
def constraint_view_plot(glyph_dict, exon_coords, var_coords, var_counts, fUTR_len, tUTR_len, chr_num, domains=[], direction='', title=''):
###right now assuming one UTR at each end of the transcript, correctly annotated
    exons_compact,vars_compact = vizcoords.adjust_coordinates(exon_coords, var_coords)    
    intron_coords = get_introns_from_exons(exon_coords)
    introns_compact = get_introns_from_exons(exons_compact)
    
    domain_coords = []
    domain_names = []
    domain_colors = []
    domains_compact = []
    domains_original = []
    
    if len(domains) > 0:
        for di in domains:
            d_start = di['start']
            d_end = di['end']
            d_name = di['name']
            d_color = di['color']
            for idx,(e_start,e_end) in enumerate(exon_coords):
                if d_start >= e_start and d_end <= e_end:
                    domain_coords.append((d_start,d_end))
                    domain_names.append(d_name)
                    domain_colors.append(d_color)
                    domains_compact.append((exons_compact[idx][0] + d_start - e_start, 
                                           exons_compact[idx][1] - (e_end - d_end)))
                    domains_original.append((d_start,d_end))
    
#     plot_height = 120
    plot_height = 200
    
    plot = figure(title=title, plot_width=1500, plot_height=plot_height,min_border=0, toolbar_location=None,
               x_range=Range1d(0, exons_compact[-1][1]), y_range=Range1d(0,30), background_fill_color='white')
    plot.grid.grid_line_color = None
        
    #COVERAGE
    for i in range(len(exon_coords)):
        coverage_glyph = add_coverage_glyph(plot, exon_coords[i], exons_compact[i], k=25)
        glyph_dict['coverage'].append(coverage_glyph)
        
    #CONSTRAINT
    max_y = -1
    for i in range(len(exon_coords)):
        expected_glyph,observed_glyph,y1s,y2s = add_constraint_glyph(plot,exon_coords[i], exons_compact[i],chr_num)   
        if max_y < max(y1s): max_y = max(y1s)
        if max_y < max(y2s): max_y = max(y2s)
        glyph_dict['constraint'].extend([expected_glyph, observed_glyph])
    
    step=10
    orig_ticks = list(range(8, 31, step))
    constraint_ticks = list(range(0, max_y+step, step))
    orig_ticks = orig_ticks[:len(constraint_ticks)]
    ticker_dict = {}
    for idx in range(len(orig_ticks)):
        ticker_dict[orig_ticks[idx]] = str(constraint_ticks[idx]/4)
    
    plot.yaxis.ticker = orig_ticks
    plot.yaxis.major_label_overrides = ticker_dict
    
    #VARIANTS
    if len(vars_compact) > 0: #avoid bokeh warning
        #indices of variations that don't fall in introns
        vars_idx = [i for i in range(len(var_coords)) if vizcoords.var_coord_in_exon_coords(var_coords[i], exon_coords)]
        vars_original = [var_coords[idx] for idx in vars_idx]
        max_count = max([var_counts[idx] for idx in vars_idx])
        var_counts = [var_counts[idx] for idx in vars_idx]        
    
        ray_glyph,circle_glyph = add_variation_glyph(plot, vars_compact, vars_original, var_counts, max_count)
        tooltips_variations = [('Position (adjusted)', '@x'), ('Position (true)', '@true_pos'), ('Count', '@counts'), ('Info', '@info')]
        plot.add_tools(HoverTool(tooltips=tooltips_variations, renderers=[circle_glyph,ray_glyph], point_policy='follow_mouse', attachment='below'))
    #
    else: 
        ray_glyph=None
        circle_glyph=None
    
    tooltips_features = [('Type','@feat_type'), ('Start (adjusted)', '@adj_start'), ('End (adjusted)', '@adj_end'), 
                      ('Start (true)', '@true_start'), ('End (true)', '@true_end'), ('Length', '@true_len')]
    
    glyph_dict['variant'].extend([ray_glyph,circle_glyph])
    
    #INTRONS    
    introns_glyph = add_intron_glyph(plot, introns_compact, intron_coords, height=14, width=15)
    plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[introns_glyph], point_policy='follow_mouse', attachment='below'))    
    
    #EXONS
    exons_glyph,arrow_glyph = add_exon_glyph(plot, exons_compact, exon_coords, direction, transcript_len=exons_compact[-1][1]-exons_compact[0][0])
    plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[exons_glyph], point_policy='follow_mouse', attachment='below'))
    
    glyph_dict['direction'].append(arrow_glyph)

    #UTRs
    tUTR_glyph=None
    if tUTR_len >= 0:
        tUTR_glyph = add_UTR_glyph(plot, [(exons_compact[0][0], exons_compact[0][0] + tUTR_len)], [(exon_coords[0][0], exon_coords[0][0] + tUTR_len)], feature_type='3\' UTR', color='#105c56')
        plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[tUTR_glyph], point_policy='follow_mouse', attachment='below'))
    
    fUTR_glyph=None
    if fUTR_len >= 0:
        fUTR_glyph = add_UTR_glyph(plot, [(exons_compact[-1][1]-fUTR_len, exons_compact[-1][1])], [(exon_coords[-1][1]-fUTR_len, exon_coords[-1][1])], feature_type='5\' UTR', color='#105c56')
        plot.add_tools(HoverTool(tooltips=tooltips_features, renderers=[fUTR_glyph], point_policy='follow_mouse', attachment='below'))
    
    glyph_dict['UTR'].extend([tUTR_glyph,fUTR_glyph])
        
    #DOMAINS
    tooltips_domains = [('Name','@domain_names'), ('Start (adjusted)', '@adj_start'), ('End (adjusted)', '@adj_end'), 
                      ('Start (true)', '@true_start'), ('End (true)', '@true_end'), ('Length', '@true_len')]
    domains_glyph = add_domain_glyph(plot, domains_compact, domains_original, domain_names, domain_colors)
    plot.add_tools(HoverTool(tooltips=tooltips_domains, renderers=[domains_glyph], point_policy='follow_mouse', attachment='below'))
    
    glyph_dict['domain'].append(domains_glyph)
    
    return plot,glyph_dict

In [203]:
#right now vcf is just a list of ints, will change to process a .vcf file
#param gff_db_path: string;
#param vcf: ; 
#param gene_name: string;
#param transcript_IDs: [string];
def constraint_view(gff_db_path, bed_path, gene_name, transcript_IDs, pfam_db_path): 
    transcript_dict = get_transcript_dict(gff_db_path, gene_name) #only doing this once during dev
#     UTR_lens_dict = get_gene_UTR_lens(gff_db_path, gene_name)
    
    #BED
    variant_ls = get_variants_bed(bed_path, gene_name=gene_name)
    var_coords = [d['pos'] for d in variant_ls]
    var_counts = [d['count'] for d in variant_ls]
    
    #MAF
#     variant_ls = get_variants_maf(maf_path)
#     var_coords = [d['start'] for d in variant_ls]
#     var_counts = [d['alt_count'] for d in variant_ls]
    #VCF
    #for VCF, it doesn't seem to have count info, so right now counts are number of mutations types, e.g. C>T, C>A
#     exons_start = exon_coords_dict['flattened-exons'][0][0]
#     exons_end = exon_coords_dict['flattened-exons'][-1][1]
#     variant_ls = get_variants_vcf('cosmic_coding_muts_chr20.vcf', start=exons_start, end=exons_end)
#     var_coords,var_counts = np.unique([variant['pos'] for variant in variant_ls], return_counts=True)
    
    exons_start = exon_dict['flattened-exons']['true_coords'][0][0]
    exons_end = exon_dict['flattened-exons']['true_coords'][-1][1]
    domains = get_domains(pfam_db_path, start=exons_start, end=exons_end)
    
    domain_names = np.unique([d['name'] for d in domains])
    domain_color_dict = {}
    for name in domain_names:
        domain_color_dict[name] = choose_random_color()
    
    for idx in range(len(domains)):
        domains[idx]['color'] = domain_color_dict[domains[idx]['name']]

    plot_ls = []
    glyph_dict = dict(UTR=[],variant=[],domain=[],direction=[],constraint=[],coverage=[])
    
    if transcript_IDs == 'all': transcript_IDs = list(exon_dict.keys())
    
    for idx,ID in enumerate(transcript_IDs):
        print(ID)
        title = 'gene={}; transcript={}'.format(gene_name, ID)
        plot,glyph_dict = constraint_view_plot(glyph_dict, transcript_dict[ID]['true_coords'], var_coords, var_counts, 
                                                          transcript_dict[ID]['tUTR_len'], transcript_dict[ID]['fUTR_len'], chr_num=transcript_dict[ID]['chr_num'], 
                                                          domains=domains, direction=transcript_dict[ID]['direction'], title=title)
        plot_ls.append(plot)
            
    legend_plot = figure(plot_height=70,plot_width=225,toolbar_location=None)
    legend_plot.yaxis.visible = legend_plot.xaxis.visible = legend_plot.grid.visible = False
    legend_glyph_expected = legend_plot.line(x=[0,1],y=[0,1],line_color=glyph_type_color('expected_mutation'))
    legend_glyph_observed = legend_plot.line(x=[0,1],y=[0,1],line_color=glyph_type_color('observed_mutation'))
    legend = Legend(items=[('Expected mutation counts', [legend_glyph_expected]), ('Observed mutation counts', [legend_glyph_observed])], location=(0,0))
    legend_glyph_expected.visible=False
    legend_glyph_observed.visible=False
    legend_plot.add_layout(legend, 'center')
    
    checkbox = CheckboxGroup(labels=['Show UTRS', 'Show domains', 'Show direction of translation', 'Show variants', 'Show mutation counts', 'Show sequencing coverage'],
                             active=[0, 1, 2, 3, 4, 5], width=100)

    callback = CustomJS(args=dict(UTR=glyph_dict['UTR'], variant=glyph_dict['variant'], domain=glyph_dict['domain'], arrow=glyph_dict['direction'], constraint=glyph_dict['constraint'], coverage=glyph_dict['coverage']), code='''
        function setVis(glyphs, vis){
            for (let i = 0; i < glyphs.length; i++){
                if(glyphs[i] != null){
                    glyphs[i].visible = vis
                }
            }
        }
        if (cb_obj.active.includes(0)) {setVis(UTR,true)}
        else {setVis(UTR,false)}
        if (cb_obj.active.includes(1)) {setVis(domain,true)}
        else {setVis(domain,false)}
        if (cb_obj.active.includes(2)) {setVis(arrow,true)}
        else {setVis(arrow,false)}
        if (cb_obj.active.includes(3)) {setVis(variant,true)}
        else {setVis(variant,false)}
        if (cb_obj.active.includes(4)) {setVis(constraint,true)}
        else {setVis(constraint,false)}
        if (cb_obj.active.includes(5)) {setVis(coverage,true)}
        else {setVis(coverage,false)}
        
        
    ''')

    checkbox.js_on_change('active', callback)
    show(column([checkbox,legend_plot] + [p for p in plot_ls]))    #TODO, option for inc. constraint or not
    #save(plot_ls[0],'test.html')

In [205]:
t0 = time.time()
constraint_view(gff_db_path, bed_path, 'KCNQ2', 'all', gtf_db_path)
#constraint_view(gff_db_path, bed_path, 'KCNQ2', ['KCNQ2-201', 'KCNQ2-202'], gtf_db_path)
t1 = time.time()
print(t1-t0)

flattened-exons
KCNQ2-201
KCNQ2-202
KCNQ2-203
KCNQ2-204
KCNQ2-205
KCNQ2-207
KCNQ2-209
KCNQ2-211
KCNQ2-213
KCNQ2-214
KCNQ2-215
KCNQ2-216
KCNQ2-217
KCNQ2-218
KCNQ2-227
KCNQ2-236


23.74174952507019
